Testbed for bots, by Arseny, based on Henry's testing_grounds.

In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json

import draftsimtools as ds

In [2]:
# Load M19 drafts
raw_drafts = ds.load_drafts("../../data/m19_2.csv")

In [3]:
# Here other folks load card lists, but I grab them from json instead
# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

with open('../../data/Allsets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
    
jsonSubset = mtgJSON['M19']['cards']

thisSet = {card['name'] : card for card in jsonSubset}
nameList = pd.DataFrame(thisSet.keys(),columns = ['Name']) # Instead of relying on cvs, get names from json

In [5]:
nameList['Name'] = nameList.Name.str.replace(' ','_')
# nameList.Name[nameList.Name.str.find(',')!=-1] # List of all names with a comma

In [6]:
# This utility method searches for "Name" column in nameList that have commas
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, as it updates both
nameList.Name[nameList.Name.str.find(',')!=-1] # There should be no longer any cards with commas

Series([], Name: Name, dtype: object)

In [7]:
# Process the drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.
Processing draft: 10000.
Processing draft: 20000.
Processing draft: 30000.
Processing draft: 40000.
Processing draft: 50000.
Processing draft: 60000.
Processing draft: 70000.
Processing draft: 80000.
Processing draft: 90000.
Processing draft: 100000.


In [15]:
# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

### Now this part below will just create weird bots and test them

But for now I can as well sketch a random bot here.

In [ ]:
# Here Henry created bots, but I do everything manually
# Creates a bot that implements current Draftsim logic
# draftsim_bot = ds.DraftsimBot(rating_dict)

In [16]:
# nameList.Name = nameList.Name.str.lower()
nCardsInSet = len(nameList)
nCardsInSet

300

In [10]:
# nameList.Name[nameList.Name.str[0]=='T']

In [17]:
a = [0, 1, 0, 1, 0, 0, 0, 0]
np.nonzero(a)[0]

array([1, 3], dtype=int64)

In [34]:
# Count-hot encoding
for iDraft in range(11,12): #range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    collection = np.zeros(nCardsInSet+1)
    for pack in draft:     
        hand = np.zeros(nCardsInSet+1)
        iCard = 0
        for card in pack:
            cardName = card
            try:
                pos = nameList[nameList.Name==cardName].index[0]
            except:
                pos = nCardsInSet # Use this very large number (300 for M19) if not found; for now
                # print("---Unrecognized card:") # All unrecognized cards here seem to be foil lands, so ignore 'em
            hand[pos] += 1
            
            if card==pack[0]: # First card, the one that was picked
                humanCard = pos
                
        # ----- that's where the bot should be called, with current collection and pack
        botCard = np.random.choice(np.nonzero(hand)[0])
        print(len(pack),botCard, humanCard==botCard)
        
        collection[humanCard] += 1

15 276 False
14 300 False
13 243 False
12 214 False
11 151 False
10 151 False
9 179 False
8 300 False
7 300 False
6 179 True
5 111 False
4 46 False
3 269 True
2 300 False
1 300 True
15 143 True
14 114 False
13 272 True
12 300 False
11 86 False
10 300 False
9 156 False
8 294 False
7 29 False
6 300 False
5 276 False
4 126 False
3 72 True
2 117 False
1 156 True
15 267 False
14 277 False
13 296 False
12 267 False
11 245 False
10 163 False
9 211 False
8 79 False
7 92 False
6 211 False
5 47 True
4 300 False
3 300 False
2 300 False
1 300 True


In [ ]:
# Gets bot accuracy for some packs
tester = ds.BotTester(test)
tester.evaluate_bots([draftsim_bot, sgd_bot], ["Draftsim2018", "Draftsim2018SGD"])
tester.write_evaluations()